In [4]:
import sys
sys.executable

'/home/goonieb/anaconda2/bin/python'

In [5]:
import os
import json
import pymongo
import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
import pickle


In [ ]:
# setup mongo
client = MongoClient()
db = client["bandstour"]
bandsintown =  db["bandsintown"]



#setup mbrainz client
musicbrainzngs.set_useragent("Example music app", "0.1", "http://example.com/music")

# source data
DATA_FOLDER = "data/labels"
countMBID = 0
countNAME = 0
count = 0
# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'
# init label_db_retrieval
labels_aliases ={}
#retrieve label for each artist before a tour starts
#i.e what is the album previously to the first year
for artist in db.minedArtists.find() :
                    print artist["name"]
    
                    #print db.minedArtists[artist]
                    #print artist
                    labels_dict= defaultdict(list)
                    labels_dict["id"]=[]

                    if  artist["mbid"] is ( None ) :
                        print ("NOMBID for",artist["name"])

                    else:
                     if  bool(artist["mbid"]) is ( False ) :
                      print ("NOMBID2 for",artist["name"])

                     else:
                      print "aartist_mbid", artist["mbid"]
                     if os.path.exists("./release_mbnz/"+artist["name"]+artist["mbid"]+".obj")==True:
                      with open("./release_mbnz/"+artist["name"]+artist["mbid"]+".obj", 'rb') as handle:
                        artist["releases"]= pickle.load(handle)
                     else:
                      try:
                        artist["releases"]=(musicbrainzngs.browse_releases( artist["mbid"], release_type=["album"], includes=["labels"] ))
                        try:
                            with open("./release_mbnz/"+artist["name"]+artist["mbid"]+".obj", 'wb') as handle:
                                pickle.dump(artist["releases"], handle, protocol=pickle.HIGHEST_PROTOCOL)    
                        except:
                            pass
                        #print "...."
                        #print artist["release_groups"]
                        key= 0
                        for release in  artist["releases"]["release-list"]:
                            #print release
                            if  release.has_key("release-event-list"):
                                #print "artist_name:",artist["name"]
                                #print "albumtitle:",release["title"]
                                #print "albumid:",release["id"]
                                #print "label-list",release["label-info-list"]
                                #print "releaseLBL",release["label-info-list"]
                                if not release["label-info-list"]:
                                    print "pppppppppppppp"
                                
                                else:
                                 if release["label-info-list"][0].has_key('label'):
                                  for label in release["label-info-list"]:
                                    #WE NEED TO TEST HERE
                                    #print "label:",label["label"]["sort-name"]
                                    #print "labelid:",label
                                    
#WE CREATE A DICTIONNARY WITH OUR FIELDS, WITH FORMAT
                                    if label.has_key('label'):
                                     if label["label"]["id"] not in labels_dict["labids"]:
                                        labels_dict["labids"].append(label["label"]["id"])
                                        labels_dict[label["label"]["id"]]={}

                                        labels_dict[label["label"]["id"]]["label_name"]=label["label"]["sort-name"]
                                        labels_dict[label["label"]["id"]]["label_id" ]=label["label"]["id"]
#WE Build the label aliases tables here so that all labels have their relationships##
                                        if label["label"]["id"] not in labels_aliases:
                                            labels_aliases[label["label"]["id"]] = musicbrainzngs.get_label_by_id(label["label"]["id"], includes=["aliases", "annotation", "area-rels", "label-rels", "place-rels"])
                                        #print "labels_aliases",labels_aliases
                                     labels_dict[label["label"]["id"]][str(key)]={}
                                    #print key
                                     labels_dict[label["label"]["id"]][str(key)]["album_id"]=release["id"]
                                     labels_dict[label["label"]["id"]][str(key)]["album_title"]=release["title"]
                                     releDate=release["release-event-list"][0]
                                #print "releDate", release["release-event-list"][0]
                                     if  releDate.has_key("date"):
                                        dateRel=arrow.get(releDate["date"])
                                        #print "date:", dateRel.strftime('%Y-%m-%d')
                                        labels_dict[label["label"]["id"]][str(key)]["album_date"]=dateRel.strftime('%Y-%m-%d')
                                     key = key +1
                                 else:
                                    print "NORELEASEDATE2"
                                    #key = key +1
                            #else:
                                #key = key +1
                                #print "NORELEASEDATE"
#NOW we make a nice lightweight object to push to mongo
                        labels_dict["name"]=artist["name"]
                        print labels_dict["name"],'\r\n'
                        db.labels_dict_per_artist.update({'myid': artist["name"] } ,labels_dict, upsert=True, multi=False)
                      except  musicbrainzngs.ResponseError:
                        print "oups",artist["name"]

                #with open(os.path.join(DATA_FOLDER,artist["name"],".json"), 'w') as outfile:
                             #   json.dump(labels_dict, outfile)
                            #db.minedArtists.artist.insert_one(labels_dict)
#with open(os.path.join(DATA_FOLDER,"label_equiv_table.json"), 'w') as outfile:
 #                               json.dump(labels_aliases, outfile)
#print labels_aliases,'\r\n\r\n\r\n'
import pickle



with open('labels_aliases', 'wb') as handle:
    pickle.dump(labels_aliases, handle, protocol=pickle.HIGHEST_PROTOCOL)



조수미
aartist_mbid dd21a142-cf44-4e22-add6-a50eea696854
Лили Иванова
aartist_mbid ab634a19-20e3-4801-975e-a230f899af8a
ᵀᴴᴱ ᴼᴿᴵᴳᴵᴻᴬᴸ Funky Monkey
('NOMBID for', u'\u1d40\u1d34\u1d31 \u1d3c\u1d3f\u1d35\u1d33\u1d35\u1d3b\u1d2c\u1d38 Funky Monkey')
Калевала
('NOMBID for', u'\u041a\u0430\u043b\u0435\u0432\u0430\u043b\u0430')
Кристина Орбакайте
aartist_mbid e71738bd-39d4-4699-b6c2-688f95b31732
Дмитрий Дмитриевич Шостакович
aartist_mbid 519dd32e-8f30-4380-8826-7aa99169e1bb
Валерия
aartist_mbid b4313680-2f57-4ec6-87a7-24873133ba4c
Евгений Игоревич Кисин
aartist_mbid 5761282a-0965-4ff6-9796-7b9ba0ed70e2
Би-2
aartist_mbid 5ca35ace-4006-4d4a-af04-5cd91dafab14
Анна Юрьевна Нетребко
aartist_mbid 90cba256-80b2-464f-9d58-a877a3757d3a
Μελίνα Ασλανίδου
aartist_mbid d55702f8-1b33-494e-8c9c-e6fd948bbee8
ŚexPistols Eẋperience
('NOMBID for', u'\u015aexPistols E\u1e8bperience')
ÜBERMENSCH (Rammstein Tribute Band)
('NOMBID for', u'\xdcBERMENSCH (Rammstein Tribute Band)')
Ørsted
('NOMBID for', u'\xd8rsted')
Øka

truett
('NOMBID for', u'truett')
transient
aartist_mbid 4cb5b170-07f9-4428-bdfa-7b74a2122553
tori kelly
('NOMBID for', u'tori kelly')
tobyMac
aartist_mbid 8484e4ab-b6b5-4326-b837-72eb72c9e8a4
tim gearan
('NOMBID for', u'tim gearan')
theo kottis
('NOMBID for', u'theo kottis')
theSHIFT
('NOMBID for', u'theSHIFT')
the vaughns
('NOMBID for', u'the vaughns')
the the Band Band
('NOMBID for', u'the the Band Band')
the peptides
('NOMBID for', u'the peptides')
the magician
('NOMBID for', u'the magician')
the Workshy
('NOMBID for', u'the Workshy')
the Whooligan
('NOMBID for', u'the Whooligan')
the Way Up
('NOMBID for', u'the Way Up')
the ThoughtCriminals
('NOMBID for', u'the ThoughtCriminals')
the Tea Club
('NOMBID for', u'the Tea Club')
the Stuntmen
('NOMBID for', u'the Stuntmen')
the Stampede
('NOMBID for', u'the Stampede')
the Sour Notes
('NOMBID for', u'the Sour Notes')
the Snips
('NOMBID for', u'the Snips')
the Sleeper
('NOMBID for', u'the Sleeper')
the Ruination
('NOMBID for', u'the Ruinat

oups yellowtieguy
xxyyxx
('NOMBID for', u'xxyyxx')
www.kierangoss.com
('NOMBID for', u'www.kierangoss.com')
wonder woman
('NOMBID for', u'wonder woman')
william becket
('NOMBID for', u'william becket')
whetherman
('NOMBID for', u'whetherman')
wax wings
('NOMBID for', u'wax wings')
w/Billy Masters on guitar
('NOMBID for', u'w/Billy Masters on guitar')
w/ This Or The Apocalypse, Ice Nine Kills
('NOMBID for', u'w/ This Or The Apocalypse, Ice Nine Kills')
w/ For Today, The Word Alive, Memphis May Fire, and Upon A Burning Body  
('NOMBID for', u'w/ For Today, The Word Alive, Memphis May Fire, and Upon A Burning Body  ')
vverevvolf
('NOMBID for', u'vverevvolf')
viech
('NOMBID for', u'viech')
uvm.
('NOMBID for', u'uvm.')
tyler boone
('NOMBID for', u'tyler boone')
trp
('NOMBID for', u'trp')
theSurrendering
('NOMBID for', u'theSurrendering')
the kats
('NOMBID for', u'the kats')
the come up
('NOMBID for', u'the come up')
the black sound
('NOMBID for', u'the black sound')
the World's Hottest '80s

oups many
mafia rusa
('NOMBID for', u'mafia rusa')
lucent dossier
('NOMBID for', u'lucent dossier')
litz 
('NOMBID for', u'litz ')
linnie & amy joy
('NOMBID for', u'linnie & amy joy')
lilabungalow
('NOMBID for', u'lilabungalow')
laCasta
('NOMBID for', u'laCasta')
la Rappresentante di Lista
('NOMBID for', u'la Rappresentante di Lista')
kojey radical
('NOMBID for', u'kojey radical')
kirk baxley
('NOMBID for', u'kirk baxley')
kachupa
('NOMBID for', u'kachupa')
k.flay
aartist_mbid d1fc999f-6184-41a6-bcb1-7c59bf74a6e1
joe mullins
('NOMBID for', u'joe mullins')
joe fox
('NOMBID for', u'joe fox')
jason philips
('NOMBID for', u'jason philips')
iwrestledabearonce, Oceano, For All Those Sleeping
('NOMBID for', u'iwrestledabearonce, Oceano, For All Those Sleeping')
it's always sunny in tijuana
('NOMBID for', u"it's always sunny in tijuana")
ici et lui
('NOMBID for', u'ici et lui')
iAROSS
('NOMBID for', u'iAROSS')
honey mustard
('NOMBID for', u'honey mustard')
hold youth
('NOMBID for', u'hold yout

Yours Tour
('NOMBID for', u'Yours Tour')
Youngstar
aartist_mbid 3e4b368a-883d-4aa3-a72f-06b938b89862
Young's Modulus
('NOMBID for', u"Young's Modulus")
Young and in the Way
('NOMBID for', u'Young and in the Way')
Young Valley
('NOMBID for', u'Young Valley')
Young Rising Sons
('NOMBID for', u'Young Rising Sons')
Young Nudy
('NOMBID for', u'Young Nudy')
Young Marco
('NOMBID for', u'Young Marco')
Young Lions
aartist_mbid b8908707-2426-4c19-bf6c-ecc87865176c
Young Frankenstein
('NOMBID for', u'Young Frankenstein')
Young Distractions
('NOMBID for', u'Young Distractions')
Young Culture
('NOMBID for', u'Young Culture')
Young Cardinals
('NOMBID for', u'Young Cardinals')
Younee
('NOMBID for', u'Younee')
You're a Good Man Charlie Brown
('NOMBID for', u"You're a Good Man Charlie Brown")
You The Few
('NOMBID for', u'You The Few')
You Man
('NOMBID for', u'You Man')
You Blew It!
('NOMBID for', u'You Blew It!')
Yotam Ben Horin
('NOMBID for', u'Yotam Ben Horin')
Yonder Mountain String Band
aartist_mbi

...

pppppppppppppp
pppppppppppppp
Murray Perahia 

Murph
aartist_mbid 9d47f849-23c5-4a9f-88eb-db2774595bf8
Murph 

Murlo
('NOMBID for', u'Murlo')
Murkage Dave
('NOMBID for', u'Murkage Dave')
Muriel 
('NOMBID for', u'Muriel ')
Murder by Death
aartist_mbid 115eda90-a9f4-4437-9e4d-513daa0d5f35
pppppppppppppp
pppppppppppppp
Murder by Death 

Murder Earth
('NOMBID for', u'Murder Earth')
Mumblr
('NOMBID for', u'Mumblr')
Multa Nox
('NOMBID for', u'Multa Nox')
Mullett
('NOMBID for', u'Mullett')
Mulatu Astatqé
aartist_mbid 67f0c7ca-14e8-47a3-be0e-bd2f2ffe25bb
pppppppppppppp
pppppppppppppp
pppppppppppppp
Mulatu Astatqé 

Mudflap Mafia
('NOMBID for', u'Mudflap Mafia')
Muck
aartist_mbid 284d9bd3-56bf-40f7-8013-47abae334b92
Muck 

Ms. Mada
('NOMBID for', u'Ms. Mada')
Ms. Fridrich
('NOMBID for', u'Ms. Fridrich')
Mr.V
('NOMBID for', u'Mr.V')
Mr.Flamboyant
('NOMBID for', u'Mr.Flamboyant')
Mr. Tophat
('NOMBID for', u'Mr. Tophat')
Mr. ROGERS
('NOMBID for', u'Mr. ROGERS')
Mr. Pink
aartist_mbid 0013cfab-4eac-

Mogwai 

Mogli
('NOMBID for', u'Mogli')
Mofro
aartist_mbid 8b36b7fa-b78c-4361-acd7-a840c0879d64
pppppppppppppp
Mofro 

Modgeist
('NOMBID for', u'Modgeist')
Modest Mouse
aartist_mbid a96ac800-bfcb-412a-8a63-0a98df600700
Modest Mouse 

Modern Saints
('NOMBID for', u'Modern Saints')
Modern Kicks
('NOMBID for', u'Modern Kicks')
Modern Enemy
('NOMBID for', u'Modern Enemy')
Modern Crowds
('NOMBID for', u'Modern Crowds')
Modena City Ramblers
aartist_mbid 2865f3f1-837b-41de-8fe2-43db57fe9a1f
pppppppppppppp
pppppppppppppp
Modena City Ramblers 

Mochester
('NOMBID for', u'Mochester')
Moby (dj set)
('NOMBID for', u'Moby (dj set)')
Mobkiss
('NOMBID for', u'Mobkiss')
Mobb Deep
aartist_mbid d75d1f08-bbb8-4eae-9877-399ca9121197
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
Mobb Deep 

Mob Rules
aartist_mbid 6883f482-3ef0-48a9-bd0a-0c0985768b25
pppppppppppppp
pppppppppppppp
Mob Rules 

Mo Lowda & The Humble
('NOMBID for', u'Mo Lowda & The Humble')
Mixtura
('

Mero 

Merle Jagger
('NOMBID for', u'Merle Jagger')
Meridian
aartist_mbid e3c6895e-6a6a-4d1d-95fa-36fae3071a0e
Meridian 

Mercy Ties
('NOMBID for', u'Mercy Ties')
Mercy Bell
('NOMBID for', u'Mercy Bell')
Mercury's Antennae
('NOMBID for', u"Mercury's Antennae")
Merchant Mariner
('NOMBID for', u'Merchant Mariner')
Mercer
aartist_mbid d6c2de6c-cc96-4f30-b581-46e395830374
Mercer 

Menno de Jong
aartist_mbid 20b2c7cc-466a-4e24-9165-141ac4caf928
Menno de Jong 

Mendonesia
('NOMBID for', u'Mendonesia')
Menage a Trois
('NOMBID for', u'Menage a Trois')
Men at Work
aartist_mbid 395cc503-63b5-4a0b-a20a-604e3fcacea2
pppppppppppppp
pppppppppppppp
pppppppppppppp
NORELEASEDATE2
Men at Work 

Memphis Rock N Soul Museum
('NOMBID for', u'Memphis Rock N Soul Museum')
Memphis
aartist_mbid 8628d49f-e6da-40e3-a8a4-d63a7d7eed96
Memphis 

Memphis May Fire
aartist_mbid 681b5ab2-b7dd-4dff-85e9-1e84503ad36a
NORELEASEDATE2
Memphis May Fire 

Memo an Miller
('NOMBID for', u'Memo an Miller')
Melting Watches
('NOMBI

Miguel 

Mighz
('NOMBID for', u'Mighz')
Mighty Moe
aartist_mbid 61a6188e-7d04-4c75-b175-7d45734cf6e1
Mighty Moe 

Mighty Maggots
('NOMBID for', u'Mighty Maggots')
Mighty Brother
('NOMBID for', u'Mighty Brother')
Midé
('NOMBID for', u'Mid\xe9')
Midwest
aartist_mbid ce030fdf-8571-49a5-aa1c-a22072a3662b
Midwest 

Midriff
('NOMBID for', u'Midriff')
Midnite
aartist_mbid 551c816c-923f-449f-b0a1-8b1ca8b3b30c
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
pppppppppppppp
Midnite 

Midnight Tyrannosaurus
('NOMBID for', u'Midnight Tyrannosaurus')
Midnight Track
('NOMBID for', u'Midnight Track')
Midnight Strangers
('NOMBID for', u'Midnight Strangers')
Midnight Snack
('NOMBID for', u'Midnight Snack')
Midnight Rodeo
('NOMBID for', u'Midnight Rodeo')
Midnight Red
('NOMBID for', u'Midnight Red')
Midnight Juggernauts
aartist_mbid 1f8e4ffd-b81b-4d10-91f4-461585bf9d16
pppppppppppppp
Midnight Juggernauts 

Midland
('NOMBID for', u'Midland')
Middle Class Rut
aartist_mbid b19576c

Michael Bublé 

Michael Brecker
aartist_mbid 795058e7-927e-4fcd-a5a0-f57c2fe9d3d1
pppppppppppppp
pppppppppppppp
Michael Brecker 

Michael Bolton
aartist_mbid e8bee5d1-a3a5-4c1a-ada4-f1ca22101e84
Michael Bolton 

Michael Barr
('NOMBID for', u'Michael Barr')
Micha
aartist_mbid d78a8556-00b6-4a12-bea5-6edf6d8e55b4
Micha 

Micah Stampley
aartist_mbid 5d242601-5550-4e41-9e75-2e4991744e3d
pppppppppppppp
pppppppppppppp
Micah Stampley 

Micah Schnabel
aartist_mbid 1bd4e7a4-800d-4df9-821a-fd23ab664621
pppppppppppppp
Micah Schnabel 

Micah McClain
('NOMBID for', u'Micah McClain')
Micah J
('NOMBID for', u'Micah J')
Mic Righteous
('NOMBID for', u'Mic Righteous')
Miami Ufo Tour
('NOMBID for', u'Miami Ufo Tour')
Miami Horror
aartist_mbid fa92553b-3cd5-4876-a7db-1d9dfcdf9fa9
Miami Horror 

Mexrrissey
('NOMBID for', u'Mexrrissey')
Mexican Gunfight
('NOMBID for', u'Mexican Gunfight')
Metropole Orchestra
('NOMBID for', u'Metropole Orchestra')
Metro Verlaine
('NOMBID for', u'Metro Verlaine')
Methods
('NO